# Profiling in C++

## Time the execution

[NaiveFibonacci.cpp](NaiveFibonacci.cpp)

In [1]:
!make naive_time

g++ -std=c++11 NaiveFibonacci.cpp -o naive


'time' is a bash function that execute a command and displays information about resources used by the command.

'-p' for results in seconds.

For more information:

In [ ]:
!man time

In [2]:
!time -p ./naive 5

5th fibonacci number: 5
real 0.00
user 0.00
sys 0.00


In [3]:
!time -p ./naive 35

35th fibonacci number: 9227465
real 0.07
user 0.07
sys 0.00


In [4]:
!time -p ./naive 40

40th fibonacci number: 102334155
real 0.71
user 0.71
sys 0.00


In [5]:
!time -p ./naive 42

42th fibonacci number: 267914296
real 1.90
user 1.89
sys 0.00


Time is increasing quicly, because of exponential complexity of code.

The real value is using the gettimeofday

Execution time is user + sys.

## Gprof2dot

Please refer to main project page : https://github.com/jrfonseca/gprof2dot

## Profiling with perf

Must be compiled with '-g' option.

In [6]:
!make naive_perf

g++ -std=c++11 -g NaiveFibonacci.cpp -o naive
perf record -g -- ./naive 42
check /proc/sys/kernel/kptr_restrict.

Samples in kernel functions may not be resolved if a suitable vmlinux
file is not found in the buildid cache or in the vmlinux path.

Samples in kernel modules won't be resolved at all.

If some relocation was applied (e.g. kexec) symbols may be misresolved
even with a suitable vmlinux or kallsyms file.

42th fibonacci number: 267914296
[ perf record: Woken up 10 times to write data ]
[ perf record: Captured and wrote 2.301 MB perf.data (~100553 samples) ]
[kernel.kallsyms] with build id af2912cf2a7fc9eab1d75c4cd0ce5c02d9cd16a7 not found, continuing without symbols
perf script | c++filt | ./gprof2dot.py -f perf | dot -Tpng -o images/output_naive_perf.png
[kernel.kallsyms] with build id af2912cf2a7fc9eab1d75c4cd0ce5c02d9cd16a7 not found, continuing without symbols


'perf' shall be launch with sudo to remove warnings and errors.
<img src="images/output_naive_perf.png">
Number of calls is not displayed.

### Native Perf User Interface

In [ ]:
!make naive_perf_bis

## Profiling with callgrind

Must be compiled with '-g' option.

In [7]:
!make naive_callgrind

g++ -std=c++11 -g NaiveFibonacci.cpp -o naive
valgrind --tool=callgrind --callgrind-out-file="callgrind.out" ./naive 38
==8306== Callgrind, a call-graph generating cache profiler
==8306== Copyright (C) 2002-2013, and GNU GPL'd, by Josef Weidendorfer et al.
==8306== Using Valgrind-3.10.1 and LibVEX; rerun with -h for copyright info
==8306== Command: ./naive 38
==8306== 
==8306== For interactive control, run 'callgrind_control -h'.
38th fibonacci number: 39088169
==8306== 
==8306== Events    : Ir
==8306== Collected : 2151735399
==8306== 
==8306== I   refs:      2,151,735,399
./gprof2dot.py --format=callgrind callgrind.out | dot -Tpng -o images/output_naive_callgrind.png


<img src="images/output_naive_callgrind.png">
Callgrind slows down significantly the execution time (x100).

### Native callgrind User Interface

In [ ]:
!kcachegrind

## Profiling with gprof

Must be compiled with '-pg' option.

In [8]:
!make naive_gprof

g++ -std=c++11 -pg NaiveFibonacci.cpp -o naive
./naive 42
42th fibonacci number: 267914296
gprof ./naive | ./gprof2dot.py | dot -Tpng -o images/output_naive_gprof.png


<img src="images/output_naive_gprof.png">
Number of fibonacci function calls is displayed : 866988873 !!

## Let's have a bad optimization idea

Let's parallelize the call of Fibonacci function

[ParallelFibonacci.cpp](ParallelFibonacci.cpp)

In [9]:
!make para_gprof

g++ ParallelFibonacci.cpp -I /usr/include/tbb -ltbb -pg -std=c++11 -o para
time -p ./para 35
35th fibonacci number: 9227465
real 8.69
user 33.91
sys 0.02
gprof ./para | ./gprof2dot.py | dot -Tpng -o images/output_para_gprof.png


<img src="images/output_para_gprof.png">

If you do not need function and template arguments information, then pass the -s in order to strip them.

If you want to keep all that information, or if the labels are still too wide, then you can pass the -w in order to wrap the labels.

You can still force displaying the whole graph by setting a zero threshold for nodes and edges via the -n0 and -e0.

## A better optimization idea

Let's iterate in the other order (memoization would be another solution).

[IterativeFibonacci.cpp](IterativeFibonacci.cpp)

In [10]:
!make iter_gprof

g++ -std=c++11 -pg IterativeFibonacci.cpp -o iter
time -p ./iter 42
42th fibonacci number: 267914296
real 0.00
user 0.00
sys 0.00
gprof ./iter | ./gprof2dot.py | dot -Tpng -o images/output_iter_gprof.png


<img src="images/output_iter_gprof.png">

## Another optimization idea

Let's use meta-programmation.

[MetaFibonacci.cpp](MetaFibonacci.cpp)

In [11]:
!make meta_gprof

g++ -std=c++11 -pg MetaFibonacci.cpp -o meta
time -p ./meta
42th fibonacci number: 267914296
real 0.00
user 0.00
sys 0.00
gprof ./meta | ./gprof2dot.py | dot -Tpng -o images/output_meta_gprof.png


<img src="images/output_meta_gprof.png">